In [ ]:
import psycopg2
from psycopg2 import sql

connection = psycopg2.connect(dbname='postgres', user='postgres', host='localhost', password='09150286678')
cursor = connection.cursor()
query = ''' CREATE DATABASE {} ;'''

name_db = ['TRADERS','USERS']

connection.autocommit = True
for i in name_db:
    cursor.execute(sql.SQL(query).format(
        sql.Identifier(i)))

cursor.close()
connection.close()

In [ ]:

import psycopg2

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

con = psycopg2.connect(dbname='postgres', user='postgres', host='localhost', password='09150286678');

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

 
cursor = con.cursor();

name_Database = "SocialMedia";

 


sqlCreateDatabase = "create database "+name_Database+";"

cursor.execute(sqlCreateDatabase)

In [ ]:
import psycopg2
from psycopg2 import Error

try:

#     connection = psycopg2.connect(user="postgres",
#                                   password="09150286678",
#                                   host="127.0.0.1",
#                                   port="5432",
#                                   database="postgres")

    connection = psycopg2.connect(dbname='hossain', user='postgres', host='localhost', password='09150286678')
    
    cursor = connection.cursor()
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

In [ ]:
from config import config

In [ ]:
import psycopg2
from config import config
from psycopg2.pool import ThreadedConnectionPool
from multiprocessing import Process
import time
import threading
from multiprocessing import Queue

data_queque = Queue()  # reader reads data from queue

SELECT_QUERY = 'Select something from some_table limit %s offset %s ';

INSERT_QUERY = "Insert INTO sometable (col1, col2, col3) values "

# writer write data to queue
class PsqlMultiThreadExample(object):
    _select_conn_count = 10;
    _insert_conn_count = 10;
    _insert_conn_pool = None;
    _select_conn_pool = None;

    def __init__(self):
        self = self;

    def postgres_connection(self):
        """ Connect to the PostgreSQL database server """
        conn = None
        try:
            # read connection parameters
            params = config()

            # connect to the PostgreSQL server
            print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(**params)

            # create a cursor
            cur = conn.cursor()

            # execute a statement
            print('PostgreSQL database version:')
            cur.execute('SELECT version()')

            # display the PostgreSQL database server version
            db_version = cur.fetchone()
            print(db_version)

            # close the communication with the PostgreSQL
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()
                print('Database connection closed.')

    def check_connection(self):
        """ Checking the postgres database connection"""
        conn = None;
        try:
            conn = PsqlMultiThreadExample._select_conn_pool.getconn()

            # create a cursor
            cur = conn.cursor()

            # execute a statement
            print('PostgreSQL database version:')
            cur.execute('SELECT version()')

            # display the PostgreSQL database server version
            db_version = cur.fetchone()
            print(db_version)
            # close the communication with the PostgreSQL
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
        finally:
            if conn is not None:
                conn.close()
                print('Database connection closed.')

    def create_connection_pool(self):
        """ Create the thread safe threaded postgres connection pool"""

        # calculate the max and min connection required
        max_conn = PsqlMultiThreadExample._insert_conn_count + PsqlMultiThreadExample._select_conn_count;
        min_conn = max_conn / 2;
        params = config()

        # creating separate connection for read and write purpose
        PsqlMultiThreadExample._insert_conn_pool = PsqlMultiThreadExample._select_conn_pool \
            = ThreadedConnectionPool(min_conn, max_conn, **params);

    def read_data(self):
        """
        This read thedata from the postgres and shared those records with each
        processor to perform their operation using threads
        Here we calculate the pardition value to help threading to read data from database

        :return:
        """
        pardition_value = 805000 / 10; # Its total record
        # this helps to identify the starting number to get data from db
        start_index = 1
        for pid in range(1, 11):
            # Getting connection from the connection pool
            select_conn = PsqlMultiThreadExample._select_conn_pool.getconn();
            insert_conn = PsqlMultiThreadExample._insert_conn_pool.getconn();
            #setting auto commit true
            insert_conn.autocommit = 1;
            # insert_conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

            #Creating 10 process to perform the operation
            ps = Process(target=self.process_data, args=(data_queque, pid, (start_index - 1) * pardition_value,
                                                               start_index * pardition_value, select_conn, insert_conn))
            start_index = start_index + 1;
            ps.daemon = True;
            ps.start();
            _start = time.time()
            ps.join()
            print "Process %s took %s seconds" % (pid, (time.time() - _start))

    def process_data(self, queue, pid, start_index, end_index, select_conn, insert_conn):
        """
        Here we process the each process into 10 multiple threads to do data process

        :param queue:
        :param pid:
        :param start_index:
        :param end_index:
        :param select_conn:
        :param insert_conn:
        :return:
        """
        print "\n";
        print(" \n Started processing record from %s to %s" % (start_index, end_index))
        pardition_value = (end_index - start_index) / 10;
        for tid in range(1, 11):
            ins_cur = insert_conn.cursor();
            worker = threading.Thread(target=self.process_thread, args=(
            queue, pid, tid, start_index, (start_index + pardition_value), select_conn.cursor(), ins_cur,
            threading.Lock()))
            start_index = start_index + pardition_value;
            worker.daemon = True;
            worker.start();
            worker.join()

    def process_thread(self, queue, pid, tid, start_index, end_index, sel_cur, ins_cur, lock):
        """
        Thread read data from database and doing the elatic search to get
        experience have the same data

        :param queue:
        :param pid:
        :param tid:
        :param start_index:
        :param end_index:
        :param sel_cur:
        :param ins_cur:
        :param lock:
        :return:
        """
        limit = end_index - start_index;
        sel_cur.execute(SELECT_QUERY,  (limit, start_index,))
        rows = sel_cur.fetchall();


        records.append(ins_cur.mogrify("(%s,%s,%s)", (row[0], row[1], row[2],)));



        self.write_data(records, ins_cur, lock)

    def write_data(self, records, ins_cur, lock):
        """
        Insert the data with experience id

        :param records:
        :param ins_cur:
        :param lock:
        :return:
        """

        lock.acquire()
        if records and records != '':
            ins_cur.execute(INSERT_QUERY + records)
        lock.release()


if __name__ == '__main__':
    _start = time.time()
    cmp_clener = PsqlMultiThreadExample();
    #Craeting database connection pool to help connection shared along process
    cmp_clener.create_connection_pool()
    cmp_clener.read_data();
    print('Total Processing time %s seconds' % (time.time() - _start))

In [ ]:
import psycopg2
for i in["USERS","TRADERS",'postgres']:
    conn = psycopg2.connect(dbname=i, user='postgres', host='localhost', password='09150286678')
    cursor = conn.cursor()
    cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
#     cursor.execute("SELECT datname FROM pg_database WHERE datistemplate = false;") # all database
    print(cursor.fetchall())


In [1]:
import psycopg2
from psycopg2.extras import RealDictCursor
conn = psycopg2.connect(dbname='postgres', user='postgres', host='localhost', password='09150286678')
postgreSQL_select_Query = "SELECT * FROM users" #WHERE token = 2
cursor= conn.cursor(cursor_factory =RealDictCursor)
cursor.execute(postgreSQL_select_Query)
colnames = [desc[0] for desc in cursor.description]
da=cursor.fetchall()
da
[{k:v for k, v in record.items()} for record in da]

[{'token': '1',
  'ip_port': 'http://www.google.com',
  'api_key': '$20',
  'api_secret': '{"name": "Google", "color": ""}',
  'api_passphrase': 'red',
  'date_registry': '8.5x11',
  'state': None,
  'is_trader': None},
 {'token': '0',
  'ip_port': 'http://www.google.com',
  'api_key': '$20',
  'api_secret': '{"name": "Google", "color": ""}',
  'api_passphrase': 'red',
  'date_registry': '8.5x11',
  'state': None,
  'is_trader': None},
 {'token': '2',
  'ip_port': 'http://www.google.com',
  'api_key': '$20',
  'api_secret': '{"name": "Google", "color": ""}',
  'api_passphrase': 'red',
  'date_registry': '8.5x11',
  'state': None,
  'is_trader': None},
 {'token': '3',
  'ip_port': 'http://www.google.com',
  'api_key': '$20',
  'api_secret': '{"name": "Google", "color": ""}',
  'api_passphrase': 'red',
  'date_registry': '8.5x11',
  'state': None,
  'is_trader': None},
 {'token': '7',
  'ip_port': 'http://www.google.com',
  'api_key': '$20',
  'api_secret': '{"name": "Google", "color": "

In [ ]:
colnames

In [ ]:
import psycopg2
import random
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from psycopg2.pool import ThreadedConnectionPool
from psycopg2 import sql
from threading import Semaphore

class ReallyThreadedConnectionPool(ThreadedConnectionPool):
    def __init__(self, minconn, maxconn, *args, **kwargs):
        self._semaphore = Semaphore(maxconn)
        super().__init__(minconn, maxconn, *args, **kwargs)

    def getconn(self, *args, **kwargs):
        self._semaphore.acquire()
        return super().getconn(*args, **kwargs)

    def putconn(self, *args, **kwargs):
        super().putconn(*args, **kwargs)
        self._semaphore.release()

tcp = ReallyThreadedConnectionPool(1,120,dbname='postgres',user='postgres',host='localhost',password='09150286678')

def write_read_to_db(tcp,i):
    thedictionary = {'name': 'Google', 'color': ''}

    try:
        conn = tcp.getconn()
        c = conn.cursor()
#         comp_schema = """
#         as_of_date DATE PRIMARY KEY NOT NULL,
#         verified_reviews INTEGER,
#         lsa_total_reviews INTEGER
#         """
#         table_name = "hossain5"

#         comp_schema ="""token VARCHAR(40) PRIMARY KEY NOT NULL,ip_port VARCHAR(40),api_key VARCHAR(40),api_secret VARCHAR(40),api_passphrase VARCHAR(40),date_registry VARCHAR(40),state VARCHAR(40),is_trader VARCHAR(40)"""
#         table_name = "users"
#         c.execute(sql.SQL("""CREATE TABLE IF NOT EXISTS %s (%s)""" %(table_name, comp_schema)))
        print(1)
        c.execute("""INSERT INTO users VALUES (%s, %s, %s, %s, %s, %s)""", 
                    (i,  'http://www.google.com', '$20', json.dumps(thedictionary), 'red', '8.5x11'))

#         stmt = """INSERT INTO mobile VALUES (%s,%s,%s)"""
#         c.execute(stmt, (all_ids2,str(all_ids2),all_ids2))
        conn.commit()
        print(1)
        tcp.putconn(conn)
        print(1)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=4) as pool:
        for i in range(1000):
            if i==1:
                continue
            t=pool.submit(write_read_to_db,tcp,i)
        

# results[0]

In [ ]:
import psycopg2
from psycopg2 import sql

column_table_USERS={"database":"USERS",
                   "users": """token VARCHAR(40) PRIMARY KEY NOT NULL,ip_port VARCHAR(40),api_key VARCHAR(40),api_secret VARCHAR(40),api_passphrase VARCHAR(40),date_registry VARCHAR(40),state VARCHAR(40),is_trader VARCHAR(40)""",
                   "alive_signal": """token VARCHAR(40),code_signal VARCHAR(40),leverag VARCHAR(40),st VARCHAR(40),sl VARCHAR(40),PNL VARCHAR(40),symbol VARCHAR(40),money VARCHAR(40),start_date VARCHAR(40)""",
                    "user_charge": """token VARCHAR(40),charge VARCHAR(40)""",
                   "archive_signal_user": """token VARCHAR(40),code_signal VARCHAR(40),leverag VARCHAR(40),st VARCHAR(40),sl VARCHAR(40),PNL VARCHAR(40),symbol VARCHAR(40),money VARCHAR(40),start_date VARCHAR(40),finnish_date VARCHAR(40)""",
                   "debat_users": """token VARCHAR(40),debat_users VARCHAR(40)""",
                   "penalty": """token VARCHAR(40),penalt VARCHAR(40)""",
                   "queue_get_signal": """token VARCHAR(40)"""}
    
column_table_TRADERS={"database":"TRADERS",
                     "creditor_trader": """token VARCHAR(40),creditor_trader VARCHAR(40)""",
                     "archive_signal_trader": """token VARCHAR(40),code_signal VARCHAR(40),leverage VARCHAR(40),st VARCHAR(40),sl VARCHAR(40),PNL VARCHAR(40),symbol VARCHAR(40),money VARCHAR(40),start_date VARCHAR(40),finnish_date VARCHAR(40)""",
                     "control_trade": """token VARCHAR(40),code_signal VARCHAR(40), type VARCHAR(40),value VARCHAR(40),date VARCHAR(40)""",
                     "penalty": """token VARCHAR(40),penalty VARCHAR(40)""",
                     "badtrade": """token VARCHAR(40),badtrade VARCHAR(40)""",
                     "rank": """token VARCHAR(40),rank VARCHAR(40),score_pnl VARCHAR(40),score_number_trade VARCHAR(40)""",
                     "expired_signal": """token VARCHAR(40),code_signal VARCHAR(40),leverage VARCHAR(40),st VARCHAR(40),sl VARCHAR(40),symbol VARCHAR(40),users VARCHAR(40),start_date VARCHAR(40)""",
                     "alive_signal": """token VARCHAR(40),code_signal VARCHAR(40),leverage VARCHAR(40),st VARCHAR(40),sl VARCHAR(40),symbol VARCHAR(40),users VARCHAR(40),start_date VARCHAR(40)"""}

all_table=[column_table_USERS,column_table_TRADERS]
for i in all_table:

    conn = psycopg2.connect(dbname=i["database"], user='postgres', host='localhost', password='09150286678')
    cur = conn.cursor()

    try:
        for j in i:
            if j=="database":
                continue
            comp_schema = i[j]
            table_name = j
            cur.execute(sql.SQL("""CREATE TABLE IF NOT EXISTS %s (%s)""" %(table_name, comp_schema)))
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    conn.commit() # <--- makes sure the change is shown in the database
    conn.close()
    cur.close()

In [ ]:
import psycopg2
from psycopg2 import Error

try:
    connection = psycopg2.connect(dbname='postgres', user='postgres', host='localhost', password='09150286678')

    cursor = connection.cursor()
    # SQL query to create a new table
    create_table_query = '''CREATE TABLE mobile
          (ID INT PRIMARY KEY     NOT NULL,
          MODEL           TEXT    NOT NULL,
          PRICE         REAL); '''
    # Execute a command: this creates a new table
    cursor.execute(create_table_query)
    connection.commit()
    print("Table created successfully in PostgreSQL ")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")